# DeltaDTM

A global coastal digital terrain model, based on CopernicusDEM, ESA WorldCover, ICESat-2 and GEDI data. For more information, see [Pronk et al. (2024)](https://www.nature.com/articles/s41597-024-03091-9) DeltaDTM: A global coastal digital terrain model. 

In [ ]:
import fsspec
import geopandas as gpd
import hvplot.xarray
import pystac
import rioxarray
import shapely
import xarray as xr
from ipyleaflet import Map, basemaps

## Read a snapshot with the spatial extents of all tiles

Connect to the CoCliCo STAC and read the spatial extents using stac-geoparquet.

In [ ]:
from coastpy.stac.utils import read_snapshot

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)

ddtm_collection = coclico_catalog.get_child("deltares-delta-dtm")

storage_options = ddtm_collection.extra_fields["item_assets"]["data"][
    "xarray:storage_options"
]

ddtm_extents = read_snapshot(
    ddtm_collection,
    columns=["geometry", "assets"],
    add_href=True,
    storage_options=storage_options,
)

ddtm_extents.head()

## Zoom to your area of interest

In [ ]:
m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = 15.827, -95.96
m.zoom = 15
m.layout.height = "800px"
m

In [ ]:
west, south, east, north = m.west, m.south, m.east, m.north
# Note: small little hack to ensure the notebook also works when running all cells at once
if not west:
    west, south, east, north = (
        30.28415679931641,
        31.276790311057272,
        30.630912780761722,
        31.51123970051334,
    )
roi = gpd.GeoDataFrame(
    geometry=[shapely.geometry.box(west, south, east, north)], crs=4326
)

## Find the tiles for your region of interest

In [ ]:
hrefs = gpd.sjoin(ddtm_extents, roi).href.to_list()
href = hrefs[0]

## Read data

In [ ]:
fs = fsspec.filesystem("az", **storage_options)
with fs.open(href, "rb") as f:
    da = (
        rioxarray.open_rasterio(f, chunks={}, lock=False)
        .squeeze()
        .drop_vars("band")
        .compute()
    )

In [ ]:
da.where(lambda xx: xx != xx.attrs["_FillValue"]).hvplot(
    x="x", y="y", geo=True, tiles="ESRI"
)

In [ ]:
def search_deltadtm_items(
    roi: gpd.GeoDataFrame,
) -> list[pystac.Item]:
    """
    Search for DeltaDTM items based on a region of interest.

    Args:
        roi (gpd.GeoDataFrame): Region of interest as a GeoDataFrame.

    Returns:
        List[pystac.Item]: List of STAC items that match the search criteria.

    Example:
        >>> roi = gpd.read_file("path_to_roi.geojson")
        >>> items = search_deltadtm_items(roi)
    """

    from coastpy.stac.utils import read_snapshot

    coclico_catalog = pystac.Catalog.from_file(
        "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
    )

    ddtm_collection = coclico_catalog.get_child("deltares-delta-dtm")

    storage_options = ddtm_collection.extra_fields["item_assets"]["data"][
        "xarray:storage_options"
    ]

    ddtm_extents = read_snapshot(
        ddtm_collection,
        columns=["id", "geometry", "assets", "href"],
        storage_options=storage_options,
    )
    return gpd.sjoin(ddtm_extents, roi).drop(columns="index_right")

In [ ]:
import pyarrow as pa

pa.Table.from_pandas(stac_table)

In [ ]:
stac_table = search_deltadtm_items(roi)
print(type(stac_table))
print(stac_table.columns)
print(stac_table)

In [ ]:
import stac_geoparquet

stac_geoparquet.to_item_collection(stac_table)

In [ ]:
def load_dem_from_items(
    items: list[pystac.Item],
    region_of_interest: gpd.GeoDataFrame,
    resampling: str | dict[str, str] | None = None,
    dtype: np.dtype | str | None = None,
    chunks: dict | None = None,
    crs: str | int | None = None,
    resolution: float | int | None = None,
    fail_on_error: bool = True,
    progress: bool | None = None,
) -> xr.Dataset:
    """
    Load DEM data from provided STAC items with flexibility for processing options.

    Args:
        items (List[pystac.Item]): STAC items to load data from.
        region_of_interest (gpd.GeoDataFrame): Region of interest as a GeoDataFrame.
        resampling (Optional[Union[str, Dict[str, str]]]): Resampling strategy. Default is "nearest".
        dtype (Optional[Union[np.dtype, str]]): Output data type. Default is "float32".
        chunks (Optional[Dict[str, Union[int, str]]]): Dask chunks for parallel loading. Default is None.
        crs (Optional[Union[str, int]]): CRS for loading. Default is "utm".
        resolution (Optional[Union[float, int]]): Spatial resolution of output. Default is 30.
        fail_on_error (bool): Skip over load failures if set to False. Default is True.
        progress (Optional[Any]): Progress bar or similar for non-Dask load. Default is None.

    Returns:
        xr.Dataset: Dataset containing the DEM data.

    Example:
        >>> items = [pystac.Item.from_file("path_to_item.json")]
        >>> roi = gpd.read_file("path_to_roi.geojson")
        >>> dem_data = load_dem_from_items(items, roi)
    """

    # STAC configuration for data loading
    STAC_CONFIG = {
        "cop-dem-glo-30": {"assets": {"*": {"data_type": dtype, "nodata": np.nan}}},
    }

    # Bounding box of the region of interest in EPSG:4326
    bbox = tuple(region_of_interest.to_crs(4326).bounds.iloc[0].values)

    # Load data using odc.stac.load with enhanced parameters
    ds = odc.stac.load(
        items,
        bbox=bbox,
        chunks=chunks,
        stac_cfg=STAC_CONFIG,
        patch_url=pc.sign,  # Placeholder, replace with actual function if needed
        resampling=resampling,
        crs=crs,
        resolution=resolution,
        fail_on_error=fail_on_error,
        progress=progress,
    ).squeeze()

    return d

In [ ]:
stac_geoparquet.from_arrow